In [ ]:
# Droplet and Vacuole Motion Analysis Notebook
#Get all the packages needed
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tifffile import TiffFile

import xml.etree.ElementTree as ET
from scipy.ndimage import center_of_mass
from skimage.draw import polygon

from ipywidgets import interact, IntSlider
from scipy.ndimage import gaussian_filter
from skimage.filters import threshold_otsu
from matplotlib.path import Path
from skimage.measure import label, regionprops, find_contours
from skimage.transform import resize
import matplotlib.cm as cm



Read all xml files in the folder

In [ ]:
#Function to read all XML files in a directory and its subdirectories
def read_all_xml_files(base_folder):
    xml_files = []
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith('.xml'):
                xml_files.append(os.path.join(root, file))
    return xml_files

In [ ]:
#List XML files from the current working directory
base_folder = os.getcwd()
xml_list = read_all_xml_files(base_folder)
print("Number of xml files detected:",len(xml_list))
print("XML files list:")
for path in xml_list:
    print(path)

Parse xml files to get roi and trajectories. 
Note that the following steps have to be done manually for every xml file

In [ ]:
xml_path = xml_list[3] #Enter the position of the xml file from the xml_list. Example here: select the 4th XML file
print(xml_path) 
#Read XML files
## Parse the TrackMate XML file
def parse_trackmate_with_roi(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    spots = {}
    tracks = []

    # Find all spots
    for spot in root.findall(".//Spot"):
        spot_id = spot.attrib['ID']
        x = float(spot.attrib['POSITION_X'])
        y = float(spot.attrib['POSITION_Y'])
        t = float(spot.attrib['FRAME'])
        radius = float(spot.attrib['RADIUS'])
        # Extract ROI points if present
        coords_text = spot.text.strip()
        coords_array = np.fromstring(coords_text, sep=' ')
        coords = coords_array.reshape(-1, 2)
        # Shift ROI coordinates by the spot's x and y
        coords_shifted = coords + np.array([x, y])
        spots[spot_id] = {"x": x, "y": y, "t": t, "radius": radius, "roi_points": coords_shifted}

    # Find all tracks
    for track in root.findall(".//Track"):
        track_id = track.attrib['TRACK_ID']
        edges = []
        for edge in track.findall(".//Edge"):
            source_id = edge.attrib['SPOT_SOURCE_ID']
            target_id = edge.attrib['SPOT_TARGET_ID']
            edges.append((source_id, target_id))
        tracks.append({"track_id": track_id, "edges": edges})

    return spots, tracks

def get_trajectories_with_roi(spots, tracks):
    trajectories = []

    for track in tracks:
        trajectory = []
        for source_id, target_id in track['edges']:
            if source_id in spots and target_id in spots:
                # print(source_id, target_id)
                trajectory.append((spots[source_id], spots[target_id]))
        trajectories.append({"track_id": track["track_id"], "trajectory": trajectory})

    return trajectories

#Here the xml file is parsed to get the spots and tracks with roi
spots, tracks = parse_trackmate_with_roi(xml_path)
trajectories = get_trajectories_with_roi(spots, tracks)
# print(trajectories)

In [ ]:
# Plot ROIs and their centroids at every timepoint in an interactive plot
all_times = sorted({spot1['t'] for traj in trajectories for spot1, _ in traj['trajectory']})
def plot_rois_at_time(t):
    plt.figure(figsize=(5, 5))
    for traj in trajectories:
        for spot1, _ in traj['trajectory']:
            # print(spot1['t'])
            if spot1['t'] == t:
                roi_points = spot1['roi_points']
                if roi_points.size > 0:
                    plt.scatter(roi_points[:, 0], roi_points[:, 1], s=1, c='pink', alpha=0.5)
                    # print(max(roi_points[:, 0]), max(roi_points[:, 1]))
                plt.scatter(spot1['x'], spot1['y'], s=10, c='blue')
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'All ROI Points at Time Point {t}')
    plt.axis('equal')
    # plt.gca().invert_yaxis()
    plt.show()

interact(plot_rois_at_time, t=IntSlider(min=int(min(all_times)), max=int(max(all_times)), step=1, value=int(min(all_times))))


In [ ]:
#find the tif file corresponding to the xml file in the current working directory
xml_filename = os.path.splitext(os.path.basename(xml_path))[0] #get the xml filename without extension
print(xml_filename)
tiff_path = None

# search inside the same "rep" folder as the xml (e.g. rep1, rep2) under the data directory
#find which "rep" folder the xml file is in
rep_name = next((p for p in xml_path.split(os.sep) if p.startswith('rep')), None)
# print(rep_name)
search_root = os.path.join(base_folder, 'data_mut', rep_name)
# print(search_root)

# walk the rep folder looking for a tif that contains the xml filename
for root_dir, dirs, files in os.walk(search_root):
    for fname in files:
        if fname.lower().endswith(('.tif', '.tiff')) and xml_filename in fname:
            tiff_path = os.path.join(root_dir, fname)
            break
    if tiff_path is not None:
        break
if tiff_path is None:
    raise FileNotFoundError(f"No .tif file found in {search_root} or {base_folder} containing '{xml_filename}'")

print("Selected TIFF:", tiff_path)

In [ ]:
# Scale image_stack frames from 2048x2048 to 682x682 (from pixel size to micron size)
with TiffFile(tiff_path) as tif: #open the tif image as an array
    image_stack = tif.asarray() 
#scaling
scaled_image_stack = np.array([
    resize(frame, (2048*0.3119631, 2048*0.3119631), preserve_range=True, anti_aliasing=True).astype(image_stack.dtype)
    for frame in image_stack
])

In [ ]:
#Plot the ROIs and their centroids from the trajectories on top of the scaled tiff image over time to double check alignment
def plot_rois_on_tiff_at_time_scaled(t):
    frame_idx = int(t)
    img = scaled_image_stack[frame_idx]
    plt.imshow(img, cmap='gray')
    for traj in trajectories:
        for spot1, _ in traj['trajectory']:
            if spot1['t'] == t:
                roi_points = spot1['roi_points']
                if roi_points.size > 0:
                    plt.plot(roi_points[:, 0], roi_points[:, 1], c='magenta', lw=0.5)
                plt.scatter(spot1['x'], spot1['y'], s=1, c='cyan')
    plt.title(f"ROIs on Scaled TIFF Frame {frame_idx}")
    plt.axis('equal')
    plt.gca().invert_yaxis()
    plt.show()

interact(plot_rois_on_tiff_at_time_scaled, t=IntSlider(min=int(min(all_times)), max=int(max(all_times)), step=1, value=int(min(all_times))))


Now we detect the vacuole area within the ROIs and track its centroid over time

In [ ]:
# Preprocess function for the image stack: Gaussian blur, Otsu thresholding, invert
def preprocess_image_stack(image_stack, sigma=1):
    # Gaussian blur
    blurred = np.array([gaussian_filter(frame, sigma=sigma) for frame in image_stack])
    # Otsu thresholding to make binary
    binary = np.array([frame > threshold_otsu(frame) for frame in blurred])
    # Invert binary image
    inverted = np.logical_not(binary)
    return inverted.astype(np.uint8)

In [ ]:
#Detect the vacuole centroids within each ROI using preprocessed images
for t in sorted({spot1['t'] for traj in trajectories for spot1, _ in traj['trajectory']}): #loop through all timepoints in the trajectories
    t_idx = int(t)
    img = scaled_image_stack[t_idx] #scale the image to micron unit
    preprocessed = preprocess_image_stack(np.expand_dims(img, 0), sigma=1)[0] #preprocess the single frame
    labeled = label(preprocessed)
    props = regionprops(labeled) #detect all regions in the preprocessed image (the image is inverted so the vacuole should be labeled)

    for traj in trajectories: #loop over each trajectory
        track_id = traj['track_id']
        for spot1, _ in traj['trajectory']:
            if spot1['t'] == t and spot1['roi_points'].size > 0: #loop over each roi at every timepoint
                roi = spot1['roi_points']
                roi_path = Path(roi)
                vacuole_centroids = []
                for prop in props:
                    y, x = prop.centroid #get the centroid of the detected region
                    #detect if the centroid is within the roi
                    if roi_path.contains_point((x, y)): 
                        if hasattr(prop, "major_axis_length") and hasattr(prop, "minor_axis_length"):
                            prop_radius = 0.5 * (prop.major_axis_length + prop.minor_axis_length) / 2
                        else:
                            prop_radius = np.sqrt(prop.area / np.pi)
                        centroid = np.array([x, y])
                        roi_radii = np.linalg.norm(roi - centroid, axis=1)
                        roi_radius = np.mean(roi_radii)
                        if prop_radius < roi_radius * 0.95:
                            vacuole_centroids.append((x, y))
                if len(vacuole_centroids) > 0:
                    spot1['vacuole_centroids'] = vacuole_centroids


In [ ]:
#Plot the vacuole centroids detected within each ROI on top of the scaled tiff image over time to double check alignment. 
# The droplet centroids and vacuole centroids are shown in the same color for each ROI
def plot_vacuole_centroids_interactive(t):
    frame_idx = int(t)
    img = scaled_image_stack[frame_idx]
    plt.figure(figsize=(8, 8))
    plt.imshow(img, cmap='gray')

    # Collect all spots at this time point
    spots_at_t = []
    for traj in trajectories:
        for spot1, _ in traj['trajectory']:
            if spot1['t'] == t:
                spots_at_t.append(spot1)
    n_spots = len(spots_at_t)
    colors = cm.get_cmap('tab20', n_spots)

    for idx, spot1 in enumerate(spots_at_t):
        color = colors(idx)
        # Plot spot centroid
        plt.scatter(spot1['x'], spot1['y'], s=25, color=color, label='Spot Centroid' if idx == 0 else "")
        # Plot vacuole centroids in the same color, smaller
        if 'vacuole_centroids' in spot1 and spot1['vacuole_centroids']:
            vac = np.array(spot1['vacuole_centroids'])
            plt.scatter(vac[:, 0], vac[:, 1], s=3, color=color, marker='o', label='Vacuole Centroid' if idx == 0 else "")

    plt.title(f"Vacuole Centroids at Frame {frame_idx}")
    plt.axis('equal')
    plt.gca().invert_yaxis()
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

all_times_sorted = sorted({spot1['t'] for traj in trajectories for spot1, _ in traj['trajectory']})

interact(
    plot_vacuole_centroids_interactive,
    t=IntSlider(min=int(min(all_times_sorted)), max=int(max(all_times_sorted)), step=1, value=int(min(all_times_sorted)))
)

In [ ]:
#Extract the vacuole centroids and roi centroids (x, y) and the time from the trajectories into a dataframe
def extract_vacuole_data(trajectories):
    data = []
    for traj in trajectories:
        track_id = traj['track_id']
        for spot1, _ in traj['trajectory']:
            if 'vacuole_centroids' in spot1 and spot1['vacuole_centroids']:
                for vac_centroid in spot1['vacuole_centroids']:
                    data.append({
                        'track_id': track_id,
                        'time': spot1['t'],
                        'roi_x': spot1['x'],
                        'roi_y': spot1['y'],
                        'vacuole_x': vac_centroid[0],
                        'vacuole_y': vac_centroid[1]
                    })
    return pd.DataFrame(data)
data = extract_vacuole_data(trajectories)

In [ ]:
#Group the data by time and track_id, and calculate the mean of the vacuole centroids
grouped_data = data.groupby(['track_id', 'time']).agg({
    'roi_x': 'mean',
    'roi_y': 'mean',
    'vacuole_x': 'mean',
    'vacuole_y': 'mean'
}).reset_index()
print(grouped_data)

In [ ]:
#Double check by plotting the mean vacuole centroids and roi centroids for each track_id at each time point on the scaled tiff image
def plot_mean_vacuole_centroids(t):
    frame_idx = int(t)
    img = scaled_image_stack[frame_idx]
    plt.figure(figsize=(8, 8))
    plt.imshow(img, cmap='gray')

    # Filter data for the current time point
    current_data = grouped_data[grouped_data['time'] == t]

    for _, row in current_data.iterrows():
        plt.scatter(row['roi_x'], row['roi_y'], s=20, color='blue')
        plt.scatter(row['vacuole_x'], row['vacuole_y'], s=10, color='red')

    plt.title(f"Mean Vacuole Centroids at Frame {frame_idx}")
    plt.axis('equal')
    plt.gca().invert_yaxis()
    plt.show()

interact(
    plot_mean_vacuole_centroids,
    t=IntSlider(min=int(min(all_times_sorted)), max=int(max(all_times_sorted)), step=1, value=int(min(all_times_sorted)))
)
#red is vacuole centroids and blue is roi centroids

Now analysis of the movement

Calculations of vectors v and x to determine angle alpha


In [ ]:
#calculate: the vector between the vacuole centroid and roi centroid for each track_id at each time and add it to the dataframe (vector v)
def calculate_vectors(data):
    data['vector_x'] = data['vacuole_x'] - data['roi_x']
    data['vector_y'] = data['vacuole_y'] - data['roi_y']
    return data
new_data = calculate_vectors(grouped_data)
# print(new_data)


In [ ]:
#Calculate vector v and x
def calculate_delta_vectors(data):
    #sort by track_id and time
    data = data.sort_values(by=['track_id', 'time'])
    # print(data)
    #calculate vector v = vector v1 (t1) - vector v1 (t0)
    data['delta_vector_x'] = data.groupby('track_id')['vector_x'].diff().fillna(0)
    data['delta_vector_y'] = data.groupby('track_id')['vector_y'].diff().fillna(0)
    #calculate vector x = roi (t1) - roi (t0)
    data['delta_roi_x'] = data.groupby('track_id')['roi_x'].diff().fillna(0)
    data['delta_roi_y'] = data.groupby('track_id')['roi_y'].diff().fillna(0)
    #calculate delta time = t1 - t0
    data['delta_time'] = data.groupby('track_id')['time'].diff().fillna(0)
    return data
delta_data = calculate_delta_vectors(new_data)
# print(delta_data)


In [ ]:
delta_data.sample(n=5)

In [ ]:
#calculate the cosine of the angle between vector v and vector x for each track_id at each time point and add it to the dataframe (calculate through the scalar product)
def calculate_cosine_angles(data):
    # Calculate length of delta_roi vectors
    data['delta_roi_length'] = (data['delta_roi_x']**2 + data['delta_roi_y']**2)**0.5
    # Calculate length of delta_vector
    data['delta_vector_length'] = (data['delta_vector_x']**2 + data['delta_vector_y']**2)**0.5
    # Calculate cosine of the angle between delta_vector and delta_roi
    data['cos_angle'] = (
        data['delta_vector_x'] * data['delta_roi_x'] +
        data['delta_vector_y'] * data['delta_roi_y']
    ) / (
        data['delta_vector_length'] * data['delta_roi_length']
    )
    return data

delta_data = calculate_cosine_angles(delta_data)


In [ ]:
delta_data.sample(n=5)

In [ ]:
#Print the delta_data dataframe to a csv file using the name of the xml file
import os
import pandas as pd 
xml_filename = os.path.basename(xml_path).replace('.xml', '')
rep = [part for part in xml_path.split(os.sep) if part.startswith('rep')][0]
delta_data.to_csv(f"{rep}_{xml_filename}_delta.csv", index=False)
print(rep, xml_filename)